# Limex Challenge - API Trading Demonstration
## SMA Crossover Trading Bot Example Using Lime Trading API

This notebook demonstrates a **Simple Moving Average (SMA) Crossover Trading Bot** using the **Lime Trading API**. The steps below outline the systematic process to automate the strategy. Users are encouraged to bring in their own strategy logic and integrate market data from their preferred vendor. This bot can also be run on a local machine, as the free version of Google Colab doesn't run continuously and may disconnect after a period of inactivity.

1. **Authenticate** with Lime Authentication service to obtain an Access Token.
2. **Fetch price data** for a chosen symbol (e.g., `AAPL`), using built-in Lime Datafeed.
3. **Calculate the 10-period and 30-period Simple Moving Averages (SMA)**.
4. **Apply crossover logic**:
   - Go **long** when the 10MA crosses **above** the 30MA.
   - **Exit any position** when the 10MA crosses **below** the 30MA.
5. **Automatically place orders** via the Lime Trading API whenever crossovers occur.
6. **Repeat every 5 minutes** to continuously check for new trading signals.

For further information about Limex Challenge competition and full API documentation, please visit:  

🔗 [Limex Challenge](https://challenges.limex.com/)

🔗 [Lime Trading API Documentation](https://docs.lime.co/trader/)

🔗 [Limex Challenge Discord Group](https://discord.com/invite/DgBEgX5J)

🔗 [Limex Challenge Rules & FAQs](https://www.promo.limex.com/faqchallenges?utm_source=email&utm_medium=d_outreach&utm_campaign=promo)

# Connecting to Lime Trading API

Import necessary libraries

In [ ]:
import requests, json
import pandas as pd
import time
from datetime import datetime

Input your credentials here

In [ ]:
# Config - Replace with your credentials
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'
USERNAME = 'your_username'
PASSWORD = 'your_password'
ACCOUNT_NUMBER = "dmo-c001@demo" #your user ID + @demo, eg: dmo-c001 + @demo

# Endpoints
AUTH_URL = 'https://auth.lime.co/connect/token'
BALANCE_URL = 'https://api.lime.co/accounts'
ORDER_URL = 'https://api.lime.co/orders/place'
PRICE_HIST_URL = 'https://api.lime.co/marketdata/history'

### Basic API functions

In [ ]:
# Get Access Token
def get_access_token():
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'password',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'username': USERNAME,
        'password': PASSWORD
    }

    response = requests.post(AUTH_URL, headers=headers, data=data)

    if response.status_code == 200:
        token_data = response.json()
        return token_data['access_token']
    else:
        raise Exception(f"Failed to get token: {response.status_code}, {response.text}")

# Fetch OHLC Data by ticker symbol
def fetch_price_data(symbol, access_token):
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    # Calculate UNIX timestamps for the last 3 days (needed for 'from' and 'to')
    now = int(time.time())
    from_time = now - (3 * 24 * 60 * 60)  # In this example - 3 days in seconds

    params = {
        'symbol': symbol,
        'period': 'minute_5',  # The supported periods are: minute, minute_5, minute_15, minute_30, hour, day, week, month, quarter, year
        'from': from_time,
        'to': now
    }

    response = requests.get(PRICE_HIST_URL, headers=headers, params=params)

    if response.status_code != 200:
        raise Exception(f"Failed to fetch price data: {response.status_code}, {response.text}")

    candles = response.json()

    # Convert candle data to a DataFrame for easy processing
    df = pd.DataFrame(candles)
    df.drop(columns=['period'], inplace=True) # clean up and show only OHLC data

    # Convert Unix timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df.set_index('timestamp', inplace=True)

    return df

# Place an Order
def place_order(access_token, order_payload):
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.post(ORDER_URL, headers=headers, json=order_payload)

    if response.status_code == 200:
        print("Order placed successfully:")
        print(response.json())
    else:
        raise Exception(f"Failed to place order: {response.status_code}, {response.text}")

# Get Account Balances
def get_account_balances(access_token):
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(BALANCE_URL, headers=headers)

    if response.status_code == 200:
        accounts = response.json()
        print("Accounts and Balances:")
        print(json.dumps(accounts, indent=2))
        return accounts
    else:
        raise Exception(f"Failed to fetch balances: {response.status_code}, {response.text}")



### Check on Connectivity and get Account Balances

In [ ]:
try:
    token = get_access_token()
    print(f"✅ Successfully connected to the Limex API server. \n\nAccess Token: {token[:15]}...")  # Quick check of first part of token

    get_account_balances(token)

except Exception as e:
    print("Error:", str(e))

# Strategy Logic

### Fetching market data

In [ ]:
# Function to fetch OHLC data and add SMA calculations to the dataframe
def fetch_current_data(symbol, access_token):
    df = fetch_price_data(symbol, access_token) # 5-min OHLC data
    df['10MA'] = df['close'].rolling(window=10).mean()
    df['30MA'] = df['close'].rolling(window=30).mean()
    return df

### Strategy logic

In [ ]:
def check_crossover(df):
    if len(df) < 32:
        return None  # Check to see if there're enough data

    latest = df.iloc[-1] #latest 'completed' bar
    prev = df.iloc[-2]

    # Long signal - 10MA crosses above 30MA
    if prev['10MA'] <= prev['30MA'] and latest['10MA'] > latest['30MA']:
        return "buy"

    # Exit signal - 10MA crosses below 30MA
    if prev['10MA'] >= prev['30MA'] and latest['10MA'] < latest['30MA']:
        return "sell"

    return None

def execute_trade(access_token, side):
    order_payload = {
        "account_number": ACCOUNT_NUMBER,
        "symbol": SYMBOL,
        "quantity": 1,
        #"price": 250.00, #price only for limit order
        "time_in_force": "day",
        "order_type": "market",
        "side": side,
        "exchange": "auto"
    }
    place_order(access_token, order_payload)

# Trading Loop (Run this to start the bot)
You can run this loop right after new 5-minute data becomes available on your datafeed. The bot will then recalculate signals every 5 minutes. If needed, you could develop the logic further to **trigger at more precise times,** and to automatically shut down once the market closes.

It is recommended to add additional safeguards, such as **automatic checks** for current holdings and position sizes, along with a **kill switch** or similar risk controls to prevent unintended trades.

In [ ]:
SYMBOL = "AAPL"
token = get_access_token()

last_signal = None  # Track the last trade signal to avoid repeat orders.
                    # Take extra care when restarting this bot
                    # - Set to None for a fresh start (no positions assumed).
                    # - Set to 'buy' if your account currently holds a long position for this strategy.

# Re-run the logic every 5 minutes to detect new trade signals (SMA crossover)
while True:
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print(f"[{current_time}] 🔄 Fetching latest market data...")

    df = fetch_current_data(SYMBOL, token)
    signal = check_crossover(df)
    if signal and signal != last_signal:
        print(f"New signal detected: {signal}")
        execute_trade(token, signal)
        last_signal = signal

    time.sleep(5 * 60) # 5 minutes in seconds